In [1]:
!git clone https://github.com/yuvalm11/hemingway.git

Cloning into 'hemingway'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 4), reused 18 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.59 MiB | 2.35 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
%cd hemingway

/content/hemingway


In [4]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.5/511.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompat

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
import torch
import wandb

In [ ]:
!wandb login

In [7]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [8]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
raw_dataset = load_dataset("json", data_files="/content/hemingway/training_data/hemingway_training_data.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
block_size = 1024

def tokenize_function(batch):
    outputs = tokenizer(
        batch["text"],
        truncation=True,
        max_length=block_size,
        padding=False,
    )
    outputs["labels"] = [ids.copy() for ids in outputs["input_ids"]]
    return outputs

In [12]:
dataset = raw_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_dataset["train"].column_names,
)

Map:   0%|          | 0/284 [00:00<?, ? examples/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    max_steps=500,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    output_dir="./hemingwrite",
    remove_unused_columns=False,
)

In [15]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [18]:
trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

/tmp/ipython-input-326993393.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
50,2.262600
100,2.198200
150,2.179400
200,2.161400
250,2.128900
300,2.137100
350,2.108700
400,2.102000
450,2.101200
500,2.094100


TrainOutput(global_step=500, training_loss=2.1473654022216797, metrics={'train_runtime': 1594.5509, 'train_samples_per_second': 1.254, 'train_steps_per_second': 0.314, 'total_flos': 1.2414324309184512e+16, 'train_loss': 2.1473654022216797, 'epoch': 7.042253521126761})

In [ ]:
gen_model = trainer.model.eval()
gen_tokenizer = tokenizer
if gen_tokenizer.pad_token is None:
    gen_tokenizer.pad_token = gen_tokenizer.eos_token

prompt = "A short story: The Sea And The Old Man\n\n"
inputs = gen_tokenizer(prompt, return_tensors="pt").to(gen_model.device)

with torch.inference_mode():
    out = gen_model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=gen_tokenizer.eos_token_id,
    )

print(gen_tokenizer.decode(out[0], skip_special_tokens=True))

A short story: The Sea And The Old Man

The sea was the old man’s friend. The sea was the old man’s friend.

He had been a good friend to the sea, but the sea had grown cruel,
and the old man had not been able to stay with it. He had gone away
and come back, and it was only when he had gone away again that he
couldn’t stay with it.

“What happened to you?” he asked the sea.

“What did I do?” the sea asked.

“I don’t know,” the old man said. “I’m not very sure. I don’t know
what happened to me.”

“You’re not very sure?”

“I don’t know. I’ve been so long away. I don’t remember. I’m not sure
what happened to me.”

“You must have been very sick,” the sea said.

